In [ ]:
print('Gajraj')

In [ ]:
import os
import sys
print(sys.version)

In [ ]:
# !python -m pip install mindsdb
# !conda list
#Following command should run on terminal and wait for some time to up and run
#!python -m mindsdb
# !pip install mindsdb
# !pip install loguru 
# !pip install pycaret

#### Register extended component (e.g Mixer,Encoder,Functionality etc..) via different code (Registration proceess i.e refer in lightwood tutorial).

#### Consume with mindsdb


In [ ]:
import pandas as pd
from lightwood.api.high_level import (
    ProblemDefinition,
    json_ai_from_problem,
    code_from_json_ai,
    predictor_from_code,
)

# Load a pandas dataset
df = pd.read_csv(
    "https://raw.githubusercontent.com/mindsdb/benchmarks/main/benchmarks/datasets/hdi/data.csv"
)

# Define the prediction task by naming the target column
pdef = ProblemDefinition.from_dict(
    {
        "target": "Development Index",  # column you want to predict
    }
)

# Generate JSON-AI code to model the problem
json_ai = json_ai_from_problem(df, problem_definition=pdef)

In [ ]:
import pandas as pd
from lightwood.api.high_level import (
    ProblemDefinition,
    json_ai_from_problem,
    code_from_json_ai,
    predictor_from_code,
)

# Load a pandas dataset
df = pd.read_csv(
    "https://raw.githubusercontent.com/mindsdb/benchmarks/main/benchmarks/datasets/hdi/data.csv"
)

# Define the prediction task by naming the target column
pdef = ProblemDefinition.from_dict(
    {
        "target": "Development Index",  # column you want to predict
    }
)

# Generate JSON-AI code to model the problem
json_ai = json_ai_from_problem(df, problem_definition=pdef)

# OPTIONAL - see the JSON-AI syntax
#print(json_ai.to_json())

# Generate python code
code = code_from_json_ai(json_ai)

# OPTIONAL - see generated code
#print(code)

# Create a predictor from python code
predictor = predictor_from_code(code)

# Train a model end-to-end from raw data to a finalized predictor
predictor.learn(df)

# Make the train/test splits and show predictions for a few examples
test_df = predictor.split(predictor.preprocess(df))["test"]
preds = predictor.predict(test_df).iloc[:10]
print(preds)

In [ ]:
df.to_csv("data.csv")

In [ ]:
print(json_ai.to_json())

In [ ]:
json_ai.model['args']['submodels'] = [{
    'module': 'random_forest_mixer.RandomForestMixer',
    'args': {
        'stop_after': '$problem_definition.seconds_per_mixer',
        'dtype_dict': '$dtype_dict',
        'target': '$target',
                'target_encoder': '$encoders[self.target]'

    }
}]

In [ ]:
print(json_ai.to_json())

In [ ]:
from lightwood.api.high_level import code_from_json_ai, predictor_from_code

code = code_from_json_ai(json_ai)
predictor = predictor_from_code(code)
predictor.learn(df)
predictions = predictor.predict(pd.DataFrame({
    'age': [63, 15, None],
    'sex': [1, 1, 0],
    'thal': [3, 1, 1]
}))
print(predictions)

### Updated extended models

In [ ]:
import pandas as pd
df=pd.read_csv('./amazon.csv')
df.head()

In [ ]:
%%writefile faker.py


from lightwood.mixer import BaseMixer
from lightwood.api.types import PredictionArguments
from lightwood.data.encoded_ds import EncodedDs, ConcatedEncodedDs
from lightwood import dtype
from lightwood.encoder import BaseEncoder
from loguru import logger
from transformers import pipeline
import torch
import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
import sys

class Faker(BaseMixer):
    # clf: RandomForestClassifier

    def __init__(self, stop_after: int, dtype_dict: dict, target: str, target_encoder: BaseEncoder):
        super().__init__(stop_after)
        self.target_encoder = target_encoder
        self.stable=True
        # Throw in case someone tries to use this for a problem that's not classification, I'd fail anyway, but this way the error message is more intuitive
        if dtype_dict[target] not in (dtype.categorical, dtype.binary):
            raise Exception(f'This mixer can only be used for classification problems! Got target dtype {dtype_dict[target]} instead!')

        # We could also initialize this in `fit` if some of the parameters depend on the input data, since `fit` is called exactly once
        # self.clf = RandomForestClassifier(max_depth=30)
        logger.add("/home/gitpod/out.log")
        self.sentiment_pipeline = pipeline("sentiment-analysis") 
        # logger.info("If you're using Python {}, prefer {feature} of course!",sys.version , feature="f-strings")

    def fit(self, train_data: EncodedDs, dev_data: EncodedDs) -> None:
        X, Y = [], []
        # By default mixers get some train data and a bit of dev data on which to do early stopping or hyper parameter optimization. For this mixer, we don't need dev data, so we're going to concat the two in order to get more training data. Then, we're going to turn them into an sklearn friendly foramat.
        logger.info(f'original data --> {ConcatedEncodedDs([train_data, dev_data]).get_column_original_data("reviewtext")}')
        for x, y in ConcatedEncodedDs([train_data, dev_data]):
            # logger.info(f'converted_train_data --> {x.tolist()}')
            X.append(x.tolist())
            Y.append(y.tolist())
            
        # self.clf.fit(X, Y)
        

    def __call__(self, ds: EncodedDs,
                 args: PredictionArguments = PredictionArguments()) -> pd.DataFrame:
        # Turn the data into an sklearn friendly format
        X = []
        # for x, _ in ds:
        #     # logger.info(f'while prediction process--> {x.tolist()}')
        #     X.append(x.tolist())

        # Yh = self.clf.predict(X)
        for item in ConcatedEncodedDs([ds]).get_column_original_data("reviewtext").tolist():       
            logger.info(f'original data for prediction--> {item}')
            X.append((1 if self.sentiment_pipeline([item])[0]['label'] == 'POSITIVE' else 0))            
        # # Lightwood encoders are meant to decode torch tensors, so we have to cast the predictions first
        # decoded_predictions = self.target_encoder.decode(torch.Tensor(Yh))

        # Finally, turn the decoded predictions into a dataframe with a single column called `prediction`. This is the standard behaviour all lightwood mixers use
        
        # logger.info(f'decoded prediction --> {decoded_predictions} , {type(decoded_predictions)}')
        # decoded_predictions=[str(i) for i in range(len(ds))]
        decoded_predictions=[str(i) for i in X]
        logger.info(f'decoded prediction --> {decoded_predictions} , {type(decoded_predictions)}')
        ydf = pd.DataFrame({'prediction': decoded_predictions})
        return ydf


In [ ]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, load_custom_module
import pandas as pd

# load the code
load_custom_module('./faker.py')

# read dataset
# df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/benchmarks/main/benchmarks/datasets/heart_disease/data.csv')

# define the predictive task
pdef = ProblemDefinition.from_dict({
    'target': 'positive', # column you want to predict
})

# generate the Json AI intermediate representation from the data and its corresponding settings
json_ai = json_ai_from_problem(df, problem_definition=pdef)

# Print it (you can also put it in a file and edit it there)
print(json_ai.to_json())

In [ ]:
json_ai.model['args']['submodels'] = [{
    'module': 'faker.Faker',
    'args': {
        'stop_after': '$problem_definition.seconds_per_mixer',
        'dtype_dict': '$dtype_dict',
        'target': '$target',
        'target_encoder': '$encoders[self.target]'
    }
}]

In [ ]:
code = code_from_json_ai(json_ai)

print(code)

# Save code to a file (Optional)
with open('sample.py', 'w') as fp:
    fp.write(code)

In [ ]:
print(json_ai.to_json())

In [ ]:
from lightwood.api.high_level import code_from_json_ai, predictor_from_code
code = code_from_json_ai(json_ai)
predictor = predictor_from_code(code)
predictor.learn(df)

In [ ]:
predictions = predictor.predict(pd.DataFrame({
    'reviewtext': ['What the hell','are you mad?','Where are you bloodyful']
}))
print(predictions)

In [ ]:
df.to_csv('amazon.csv')

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
data = ["This is a pretty good version of the game for being free. There are LOTS of different levels to play. My kids enjoy it a lot too."]
data = [data[0].split(".")[0]]
data=(1 if sentiment_pipeline(data)[0]['label'] == 'POSITIVE' else 0)
data

In [ ]:
df.reviewtext=df.reviewtext.apply(lambda x : x.split(".")[0])

In [ ]:
df.to_csv("amazon.csv")